In [50]:
# import libraries
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import concurrent.futures
from tqdm import tqdm
from datetime import datetime
from pathlib import Path
sys.path.append('/home/dong1128/deepQPF_preparation')

import src.data_utils as du

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
def get_filename_from_filename(fpath: str):
    token = fpath.split('/')[-1]
    ymd = token.split('.')[1]
    hm = token.split('.')[2]

    dt = datetime(int(ymd[:4]), int(ymd[4:6]), int(ymd[6:]), int(hm[:2]), int(hm[2:]))
    day_dir = os.path.join(
        '/work/dong1128/database/radar_2d_cropped/',
        str(dt.year),
        f'{dt.year}{dt.month:02}',
        f'{dt.year}{dt.month:02}{dt.day:02}',
    )
    os.makedirs(day_dir, exist_ok=True)
    new_fpath = os.path.join(day_dir, ymd+'_'+hm+'.nc')
    return new_fpath

def main(file_path: Path):
    lon = np.linspace(118, 123.5, 441)
    lat = np.linspace(20, 27, 561)
    d_sparse = np.loadtxt(file_path, dtype=np.float32)
    data = np.zeros([561, 441], dtype=np.float32)
    if len(d_sparse) == 0:
        pass
    elif len(d_sparse) == 3 and len(d_sparse.shape) == 1:
        data[d_sparse[0].astype(np.int32), d_sparse[1].astype(np.int32)] = d_sparse[2]
    else:
        data[d_sparse[0, :].astype(np.int32), d_sparse[1, :].astype(np.int32)] = d_sparse[2, :]
    
    fname = get_filename_from_filename(str(file_path))
    du.save_nc(fname, data, 'cv', (561, 441), lat, lon)

In [52]:
file_dir = Path('/work/dong1128/database/radar_2d_cropped_sparse/')
file_paths = sorted(file_dir.rglob('*.gz'))

with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    list(tqdm(executor.map(main, file_paths), total=len(file_paths)))
    

  0%|          | 231/209359 [00:02<40:27, 86.14it/s]/tmp/ipykernel_3117/2045914124.py:20: UserWarning: loadtxt: Empty input file: "/work/dong1128/database/radar_2d_cropped_sparse/2015/201501/20150102/MREF3D21L.20150102.1530.gz"
  d_sparse = np.loadtxt(file_path, dtype=np.float32)
/tmp/ipykernel_3117/2045914124.py:20: UserWarning: loadtxt: Empty input file: "/work/dong1128/database/radar_2d_cropped_sparse/2015/201501/20150102/MREF3D21L.20150102.1640.gz"
  d_sparse = np.loadtxt(file_path, dtype=np.float32)
  0%|          | 249/209359 [00:03<44:13, 78.81it/s]/tmp/ipykernel_3117/2045914124.py:20: UserWarning: loadtxt: Empty input file: "/work/dong1128/database/radar_2d_cropped_sparse/2015/201501/20150102/MREF3D21L.20150102.1940.gz"
  d_sparse = np.loadtxt(file_path, dtype=np.float32)
  0%|          | 259/209359 [00:03<42:17, 82.41it/s]/tmp/ipykernel_3117/2045914124.py:20: UserWarning: loadtxt: Empty input file: "/work/dong1128/database/radar_2d_cropped_sparse/2015/201501/20150102/MREF3D21L